In [2]:
#!/usr/bin/env python3
import os
import numpy
from numpy import random

# import scipy
import matplotlib
import pickle

# matplotlib.use("agg")
from matplotlib import pyplot as plt

import torch
import torchvision

# you may wish to import other things like nn
from torch.utils.data import DataLoader
import torch.nn as nn

# hyperparameter settings and other constants
batch_size = 100
num_classes = 10
epochs = 10
mnist_input_shape = (28, 28, 1)
d1 = 1024
d2 = 256
alpha = 0.1
beta = 0.9
alpha_adam = 0.001
rho1 = 0.99
rho2 = 0.999
# end hyperparameter settings


# load the MNIST dataset using TensorFlow/Keras
def load_MNIST_dataset():
    train_dataset = torchvision.datasets.MNIST(
        root="./data",
        train=True,
        transform=torchvision.transforms.ToTensor(),
        download=True,
    )
    test_dataset = torchvision.datasets.MNIST(
        root="./data",
        train=False,
        transform=torchvision.transforms.ToTensor(),
        download=False,
    )
    return (train_dataset, test_dataset)


# construct dataloaders for the MNIST dataset
#
# train_dataset        input train dataset (output of load_MNIST_dataset)
# test_dataset         input test dataset (output of load_MNIST_dataset)
# batch_size           batch size for training
# shuffle_train        boolean: whether to shuffle the training dataset
#
# returns              tuple of (train_dataloader, test_dataloader)
#     each component of the tuple should be a torch.utils.data.DataLoader object
#     for the corresponding training set;
#     use the specified batch_size and shuffle_train values for the training DataLoader;
#     use a batch size of 100 and no shuffling for the test data loader


def construct_dataloaders(train_dataset, test_dataset, batch_size, shuffle_train=True):
    train_dataloader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=shuffle_train
    )
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_dataloader, test_dataloader


# evaluate a trained model on MNIST data
#
# dataloader    dataloader of examples to evaluate on
# model         trained PyTorch model
# loss_fn       loss function (e.g. nn.CrossEntropyLoss)
#
# returns       tuple of (loss, accuracy), both python floats
@torch.no_grad()
def evaluate_model(dataloader, model, loss_fn):
    model.eval()
    loss = correct = total = 0
    for inputs, labels in dataloader:
        output = model(inputs)
        loss += len(labels) * loss_fn(output, labels).item()
        correct += sum(torch.argmax(output, dim=1) == labels)
        total += output.shape[0]
    avg_loss = loss / total
    acc = correct / total
    return avg_loss, acc


def make_fully_connected_model_part1_1():
    return nn.Sequential(
        nn.Flatten(),
        nn.LazyLinear(out_features=1024),
        nn.ReLU(),
        nn.Linear(in_features=1024, out_features=256),
        nn.ReLU(),
        nn.Linear(in_features=256, out_features=10),
    )

    # build a fully connected two-hidden-layer neural network with Batch Norm, as in Part 1.4
    # use the default initialization for the parameters provided in PyTorch
    # returns   a new model of type nn.Sequential


def make_fully_connected_model_part1_4():
    return nn.Sequential(
        nn.Flatten(),
        nn.LazyLinear(out_features=1024),
        nn.BatchNorm1d(1024),
        nn.ReLU(),
        nn.Linear(in_features=1024, out_features=256),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.Linear(in_features=256, out_features=10),
    )


# build a convolutional neural network, as in Part 3.1
# use the default initialization for the parameters provided in PyTorch
#
# returns   a new model of type nn.Sequential


def make_cnn_model_part3_1():
    return nn.Sequential(
        nn.LazyConv2d(16, (3, 3), 1, 0),
        nn.LazyBatchNorm2d(),
        nn.ReLU(),
        nn.LazyConv2d(16, (3, 3), 1, 0),
        nn.LazyBatchNorm2d(),
        nn.ReLU(),
        nn.MaxPool2d((2, 2)),
        nn.LazyConv2d(32, (3, 3), 1, 0),
        nn.LazyBatchNorm2d(),
        nn.ReLU(),
        nn.MaxPool2d((2, 2)),
        nn.Flatten(),
        nn.LazyLinear(out_features=128),
        nn.ReLU(),
        nn.Linear(128, 10),
    )

    # train a neural network on MNIST data
    #     be sure to call model.train() before training and model.eval() before evaluating!
    #
    # train_dataloader   training dataloader
    # test_dataloader    test dataloader
    # model              dnn model to be trained (training should mutate this)
    # loss_fn            loss function
    # optimizer          an optimizer that inherits from torch.optim.Optimizer
    # epochs             number of epochs to run
    # eval_train_stats   boolean; whether to evaluate statistics on training set each epoch
    # eval_test_stats    boolean; whether to evaluate statistics on test set each epoch
    #
    # returns   a tuple of
    #   train_loss       an array of length `epochs` containing the training loss after each epoch, or [] if eval_train_stats == False
    #   train_acc        an array of length `epochs` containing the training accuracy after each epoch, or [] if eval_train_stats == False
    #   test_loss        an array of length `epochs` containing the test loss after each epoch, or [] if eval_test_stats == False
    #   test_acc         an array of length `epochs` containing the test accuracy after each epoch, or [] if eval_test_stats == False
    #   approx_tr_loss   an array of length `epochs` containing the average training loss of examples processed in this epoch
    #   approx_tr_acc    an array of length `epochs` containing the average training accuracy of examples processed in this epoch

from tqdm import tqdm
def train(
    train_dataloader,
    test_dataloader,
    model,
    loss_fn,
    optimizer,
    epochs,
    eval_train_stats=True,
    eval_test_stats=True,
):
    train_loss, train_acc, test_loss, test_acc, approx_tr_loss, approx_tr_acc = (
        [],
        [],
        [],
        [],
        [],
        [],
    )
    for _ in tqdm(range(epochs)):
        model.train()
        total_loss = total_correct = total_examples = 0
        for batch_input, batch_labels in train_dataloader:
            optimizer.zero_grad()
            preds = model(batch_input)
            loss = loss_fn(preds, batch_labels)
            loss.backward()
            optimizer.step()
            total_loss += len(batch_labels) * loss.item()
            total_correct += sum(torch.argmax(preds, dim=1) == batch_labels)
            total_examples += len(batch_labels)
        approx_tr_loss.append(total_loss / total_examples)
        approx_tr_acc.append(total_correct / total_examples)
        if eval_train_stats:
            loss, acc = evaluate_model(train_dataloader, model, loss_fn)
            train_loss.append(loss)
            train_acc.append(acc)
        if eval_test_stats:
            loss, acc = evaluate_model(test_dataloader, model, loss_fn)
            test_loss.append(loss)
            test_acc.append(acc)
    return (train_loss, train_acc, test_loss, test_acc, approx_tr_loss, approx_tr_acc)



In [3]:
(train_dataset, test_dataset) = load_MNIST_dataset()
train_dataloader, test_dataloader = construct_dataloaders(
    train_dataset, test_dataset, 100
)


100%|██████████| 9912422/9912422 [00:00<00:00, 139331592.61it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 108156869.49it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 35698976.90it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 18057373.24it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
# SGD with Momentum
epochs = 10
lrs = [1.0,0.3,0.1,0.03,0.01,0.003,0.001]
for lr in lrs:
  model_1_2 = make_fully_connected_model_part1_1()
  cross_entropy = nn.CrossEntropyLoss()
  momentum_opt = torch.optim.SGD(model_1_2.parameters(), lr=lr, momentum=0.9)
  train_loss, train_acc, test_loss, test_acc, approx_tr_loss, approx_tr_acc = train(
      train_dataloader, test_dataloader, model_1_2, cross_entropy, momentum_opt, epochs,
      eval_train_stats=False,
    eval_test_stats=False,
  )
  test_loss, test_acc = evaluate_model(test_dataloader, model_1_2, cross_entropy)
  print(f'Learning Rate: {lr}')
  print(f'Val Loss: {test_loss}, Val Accuracy: {test_acc}')
  print(f'Moving on to next one...')



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
100%|██████████| 10/10 [04:45<00:00, 28.54s/it]


Learning Rate: 1.0
Val Loss: 2.3252865505218505, Val Accuracy: 0.10279999673366547
Moving on to next one...


100%|██████████| 10/10 [04:10<00:00, 25.00s/it]


Learning Rate: 0.3
Val Loss: 0.14741767410472675, Val Accuracy: 0.9682999849319458
Moving on to next one...


100%|██████████| 10/10 [03:32<00:00, 21.21s/it]


...

Learning Rate: 0.01
Val Loss: 0.06534248367184774, Val Accuracy: 0.9801999926567078
Moving on to next one...


100%|██████████| 10/10 [03:13<00:00, 19.34s/it]


Learning Rate: 0.003
Val Loss: 0.1296159452246502, Val Accuracy: 0.961899995803833
Moving on to next one...


100%|██████████| 10/10 [03:07<00:00, 18.70s/it]


Learning Rate: 0.001
Val Loss: 0.2455887557566166, Val Accuracy: 0.9305999875068665
Moving on to next one...


In [5]:
def batch_norm(width = 1024, narrow_width = 256):
    return torch.nn.Sequential(
        torch.nn.Flatten(),
        torch.nn.LazyLinear(out_features=width),
        torch.nn.BatchNorm1d(width),
        torch.nn.ReLU(),
        torch.nn.Linear(in_features=width, out_features=narrow_width),
        torch.nn.BatchNorm1d(narrow_width),
        torch.nn.ReLU(),
        torch.nn.Linear(in_features=narrow_width, out_features=10),
		torch.nn.BatchNorm1d(10)
    )


In [9]:
# Perform random search
best_score = float('-inf')
best_hyperparameters = None
num_trials = 10
lr_bound = [0,0.1]
beta_bound = [0.8,1]
width_bound = [512, 2048]
narrow_bound = [128, 512]
from time import time
start = time()

for _ in range(num_trials):
    learning_rate = random.uniform(lr_bound[0],lr_bound[1])
    beta = random.uniform(beta_bound[0],beta_bound[1])
    width = random.randint(width_bound[0],width_bound[1])
    narrow = random.randint(narrow_bound[0],narrow_bound[1])
    model_1_4 = batch_norm(width=width, narrow_width=narrow)
    cross_entropy = torch.nn.CrossEntropyLoss()
    momentum_opt = torch.optim.SGD(
        model_1_4.parameters(), lr=learning_rate, momentum=beta)
    train_loss, train_acc, test_loss, test_acc, approx_tr_loss, approx_tr_acc = train(
        train_dataloader, test_dataloader, model_1_4, cross_entropy, momentum_opt, epochs,
          eval_train_stats=False,
      eval_test_stats=False,
    )
    test_loss, test_acc = evaluate_model(test_dataloader, model_1_4, cross_entropy)
    print(f'lr: {learning_rate}, beta: {beta}, width: {width}, narrow: {narrow}')
    print(f'Val Loss: {test_loss}, Val Accuracy: {test_acc}')
    print(f'Moving on to next one...')

    if test_acc > best_score:
        best_score = test_acc
        best_hyperparameters = (learning_rate, beta, width, narrow)
final = time()
difference = final - start
print(time)
print("Best hyperparameters:")
print("Learning Rate:", best_hyperparameters[0])
print("Beta:", best_hyperparameters[1])
print("Width:", best_hyperparameters[2])
print("Narrow:", best_hyperparameters[3])


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
100%|██████████| 10/10 [02:12<00:00, 13.23s/it]


lr: 0.01674814374014366, beta: 0.9834734024541815, width: 1293, narrow: 170
Val Loss: 0.05530764216899115, Val Accuracy: 0.9857000112533569
Moving on to next one...


100%|██████████| 10/10 [02:50<00:00, 17.04s/it]


lr: 0.042571575457360025, beta: 0.81840533445735, width: 1547, narrow: 348
Val Loss: 0.051586740053026006, Val Accuracy: 0.9855999946594238
Moving on to next one...


100%|██████████| 10/10 [01:56<00:00, 11.64s/it]


...

lr: 0.06414799715802426, beta: 0.972671549222937, width: 1650, narrow: 260
Val Loss: 0.05369049780962087, Val Accuracy: 0.984499990940094
Moving on to next one...


100%|██████████| 10/10 [02:04<00:00, 12.40s/it]


lr: 0.023120842423648636, beta: 0.912110449445107, width: 843, narrow: 323
Val Loss: 0.04360672566312132, Val Accuracy: 0.9866999983787537
Moving on to next one...


100%|██████████| 10/10 [01:45<00:00, 10.58s/it]


lr: 0.0017033288965179084, beta: 0.9359535177919398, width: 527, narrow: 355
Val Loss: 0.062222722796723244, Val Accuracy: 0.9829000234603882
Moving on to next one...
<built-in function time>
Best hyperparameters:
Learning Rate: 0.023120842423648636
Beta: 0.912110449445107
Width: 843
Narrow: 323


In [13]:
print(f'Time: {difference}')

Time: 1374.897782087326


In [ ]:
betas = [0.95,0.9,0.85, 0.75]
widths = [4096, 2048, 1024, 512]
narrow_widths = [512, 256, 128, 64]
# batch norm
epochs = 10
for beta in betas:
  for width in widths:
    for narrow in narrow_widths:
      model_1_4 = batch_norm(width=width, narrow_width=narrow)
      cross_entropy = torch.nn.CrossEntropyLoss()
      momentum_opt = torch.optim.SGD(
          model_1_4.parameters(), lr=.001, momentum=beta)
      train_loss, train_acc, test_loss, test_acc, approx_tr_loss, approx_tr_acc = train(
          train_dataloader, test_dataloader, model_1_4, cross_entropy, momentum_opt, epochs,
            eval_train_stats=False,
        eval_test_stats=False,
      )
      test_loss, test_acc = evaluate_model(test_dataloader, model_1_4, cross_entropy)
      print(f'beta: {beta}, width: {width}, narrow: {narrow}')
      print(f'Val Loss: {test_loss}, Val Accuracy: {test_acc}')
      print(f'Moving on to next one...')




/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
100%|██████████| 10/10 [10:08<00:00, 60.86s/it]


beta: 0.95, width: 4096, narrow: 512
Val Loss: 0.06053978884592652, Val Accuracy: 0.9832000136375427
Moving on to next one...


100%|██████████| 10/10 [07:36<00:00, 45.66s/it]


beta: 0.95, width: 4096, narrow: 256
Val Loss: 0.059152940274216234, Val Accuracy: 0.9843999743461609
Moving on to next one...


100%|██████████| 10/10 [06:48<00:00, 40.86s/it]


...

100%|██████████| 10/10 [07:09<00:00, 42.98s/it]


beta: 0.85, width: 4096, narrow: 128
Val Loss: 0.09037665404379368, Val Accuracy: 0.9818999767303467
Moving on to next one...


100%|██████████| 10/10 [06:45<00:00, 40.58s/it]


beta: 0.85, width: 4096, narrow: 64
Val Loss: 0.09163711076602339, Val Accuracy: 0.980400025844574
Moving on to next one...


 10%|█         | 1/10 [00:34<05:07, 34.15s/it]